# 1. Identifying fraudulent claims

You have a project with a new customer who has asked you to find a way to prioritize suspicious claims on the motor line of business, in order to help the investigators of the company on their daily work.

You have already started the project with the customer and after some initial meetings it has been possible to identify what data is needed and what data is available. Finally they have delivered to you the data in a csv file called claims.csv.



# 2. Data

The data that you will find here has the following information

|| |||
| --- ||||
| monthsascustomer | age | policy_number | policybinddate |
| policy_state | policy_csl | policy_deductable | policyannualpremium |
| umbrella_limit | insured_zip | insured_sex | insurededucationlevel |
| insured_occupation | insured_hobbies | insured_relationship | capital-gains |
| capital-loss | incident_date | incident_type | collision_type |
| incident_severity | authorities_contacted | incident_state | incident_city |
| incident_location | incidenthouroftheday | numberofvehicles_involved | property_damage |
| bodily_injuries | witnesses | policereportavailable | totalclaimamount |
| injury_claim | property_claim | vehicle_claim | auto_make |
| auto_model | auto_year | fraud_reported | _c39] |

# 3. Solution

Now is your turn. In the next step you need to develop a prototype with this data and show the customer your results and try to convince them that your model will offer an added value to the company. Good luck!!

---

The target variable is `fraud_reported`, which can assume a value in {0,1}.

The purpose of the model is to prioritize suspicious claims; we expect the target variable to be unbalanced with relatively few frauds. <br/>
Thus, we intend to use logistic regression, to obtain a probability of fraud p∈[0,1] for each sample.

Before implementing any model, we focus on feature analysis and feature engineering. <br/>
We start by examining the percentage of missing values in each column.

In [1]:
import pandas as pd
from IPython.display import display, Markdown as md


In [2]:
INPUT_FILE = "claims.csv"
df = pd.read_csv(INPUT_FILE)
y_name = "fraud_reported"

### 3.1 Missing values

In [3]:
# What is the percentage of missing values in each column?
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum() / df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
_c39,700,1.0
age,0,0.0
incident_state,0,0.0
incident_city,0,0.0
incident_location,0,0.0
incident_hour_of_the_day,0,0.0
number_of_vehicles_involved,0,0.0
property_damage,0,0.0
bodily_injuries,0,0.0
witnesses,0,0.0


We observe that only one column has 100% missing values and can be dropped. 

In [4]:
columns_to_drop = (missing_data[missing_data['Total'] > 10]).index
df = df.drop(columns_to_drop, axis=1)

### 3.2 Variable modifications

Which values does each column hold? <br/>
We could find 'NA' values equivalent to null values, or columns that contain the same value in 100% of the samples

Number of samples in the dataset : 700 <br/>
For each variable, we display the (max 10) most frequent values in [value, frequency] format:

In [5]:
for c in df.columns:
    vc = df[c].value_counts(ascending=False).iloc[0:10]
    display(pd.DataFrame(vc))

,months_as_customer
289,6
259,6
230,6
210,6
140,6
285,5
107,5
128,5
156,5
245,5


,age
34,37
43,33
33,32
41,31
39,29
31,29
32,28
30,28
37,27
38,26


,policy_number
125591,1
836349,1
130930,1
413192,1
226725,1
533941,1
349658,1
756870,1
942106,1
432740,1


,policy_bind_date
2002-07-16,2
1997-02-03,2
1992-08-05,2
1995-09-19,2
2008-05-16,2
2002-12-28,2
1991-07-20,2
1995-12-07,2
1999-12-07,2
2006-01-01,2


,policy_state
OH,249
IL,237
IN,214


...

,vehicle_claim
5040,6
44800,5
3600,5
33600,5
3360,4
25200,3
2940,3
46000,3
4340,3
4720,3


,auto_make
Suburu,57
Nissan,56
Saab,56
Toyota,55
Chevrolet,54
Volkswagen,54
Audi,51
Ford,49
Dodge,49
BMW,48


,auto_model
Wrangler,31
RAM,29
A3,28
Passat,28
Jetta,26
A5,23
Legacy,23
Pathfinder,23
MDX,23
Forrestor,22


,auto_year
1995,42
2002,41
2009,40
2006,38
2005,38
2003,36
2013,36
1999,36
2000,35
2011,34


,fraud_reported
0,519
1,181


We drop the following columns:
- `policy_number`: since it constitutes an ID
- `incident_location`: the incident may have happened at the same address in a different city or US state 
- `insured_zip`: there is a different ZIP for each sample; we are already using different geographical information (state, city)
- `insured_hobbies`: it is a detail of dubious relevance at a very high level of granularity, we keep the rest of the information about the customer (education, occupation)
- potentially, the columns policy_state, insured_zip, incident_city, auto_model can be dropped using the same reasoning
- `policy_bind_date`, `incident_date`: replaced by `insurance_duration`

We turn several variables from categorical to numerical, because there is information in the order :
- `policy_csl`: {100/300, 250/500, 500/1000}, it makes more sense as a numerical variable
- `incident_severity`: {Trivial/Minor/Major Damage, Total Loss},
- `police_report_available`: NO=0, ?=1, YES=2
- `property_damage`: NO=0, ?=1, YES=2

We create the following columns:
- `insurance_duration`: expressed in days, it replaces the variables `policy_bind_date` and `incident_date`



In [6]:
df_1 = df.drop(["policy_number", "incident_location", "insured_zip", "insured_hobbies"], axis=1)

a = pd.to_datetime(df["incident_date"], yearfirst=True)
b = pd.to_datetime(df["policy_bind_date"], yearfirst=True)
policy_time = a - b
df_1["insurance_duration"] = policy_time
df_1 = df_1.drop(["incident_date", "policy_bind_date"], axis=1)

df_1 = df_1.replace({"policy_csl": {"100/300": 1, "250/500": 2, "500/1000": 3},
                     "incident_severity": {"Trivial Damage": 0, "Minor Damage": 1, "Major Damage": 2, "Total Loss": 3},
                     "police_report_available": {"NO": 0, "?": 1, "YES": 2},
                     "property_damage": {"NO": 0, "?": 1, "YES": 2}})
df = df_1

Before encoding the categorical variables, we examine the number of unique values (i.e. categories) for each one. <br/>
We should be aware of the expansion of the number of features that derives from applying a one-hot encoder, and reflect upon which variables are worth keeping.

In [7]:
lts = []
for c in df.columns:
    if df[c].dtype == pd.CategoricalDtype:  # we don't need to examine numerical variables here
        lts.append((c, df[c].value_counts().count()))
unique_values_df = pd.DataFrame(lts, columns=["Variable", "n. unique values"])
display(unique_values_df)

,Variable,n. unique values
0,policy_state,3
1,insured_sex,2
2,insured_education_level,7
3,insured_occupation,14
4,insured_relationship,6
5,incident_type,4
6,collision_type,4
7,authorities_contacted,5
8,incident_state,7
9,incident_city,7
